# Part 2: Modelling 

In [275]:
import os
home_path =  os.path.expanduser('~')
os.chdir(home_path + '\Documents\Github\datascience_1')
!git pull

Already up-to-date.


In [276]:
import sys
sys.path.append(home_path + '\Documents\Github\datascience_1')
from week7 import *
%who function

accuracy	 build_pred	 build_tree_iter	 caser	 closest_centroid	 compute_mean	 compute_prediction	 compute_training	 euclidean_distance	 
f1	 find_best_splitter	 forest_builder	 generate_table	 get_oob_table	 gig	 gini	 informedness	 initialize_centroids	 
k_fold	 k_fold_random	 k_means	 multi_find_best_splitter	 multi_gig	 multi_gini	 new_seed	 oob_vote_taker	 phase_1	 
phase_2	 pitch_build_tree_iter	 pitch_compute_prediction	 pred_accuracy	 predictor_case	 probabilities	 row_to_vect	 seeder	 tree_predictor	 
unanimous_rows	 vote_taker	 


In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)
pitcher = pd.read_csv('~\Documents\CS\CIS399\greinke_part1_done.csv') 

# Drop unused columns

These are the columns for K-means, which will not be included in the decision tree process.

In [5]:
pitcher = pitcher.drop(['pitch_CHkmeans','pitch_SLkmeans','pitch_FFkmeans','pitch_FTkmeans','pitch_FBkmeans','pitch_CUkmeans'],axis=1)

# Modfiy Functions for Multinomial Response

The function that was originally used to compute the prediction for a give path of the decision tree only gave binomial responses, whereas for my use I will need four response variables. I modified the prediciton function below and added this helper function to the main function to build the decision tree and the function to create a random forest.

In [6]:
def pitch_compute_prediction(table, target):
    counts = [sum(table['pitch'] == 0),sum(table['pitch'] == 1),sum(table['pitch'] == 2),sum(table['pitch'] == 3)]
    max_val = max(counts)
    if max_val == 0:
        raise LookupError('Prediction impossible - Empty tree on leaf')
    for i in range(len(counts)):
        if counts[i] == max_val:
            return i
        else:
            continue
  

In [285]:
def multi_probabilities(counts):
    count_0 = 0 if 0 not in counts else counts[0]  #could have no 0 values
    count_1 = 0 if 1 not in counts else counts[1]
    count_2 = 0 if 2 not in counts else counts[2]
    count_3 = 0 if 3 not in counts else counts[3]
    total = count_0 + count_1 + count_2 + count_3
    probs = (0,0,0,0) if total == 0 else (1.0*count_0/total, 1.0*count_1/total,1.0*count_2/total,1*0*count_3/total)  #build 2-tuple
    return probs

In [286]:
def multi_gini(counts):
    (p0,p1,p2,p3) = probabilities(counts)
    sum_probs = p0**2 + p1**2 + p2**2 + p3**2
    gini = 1 - sum_probs
    return gini

In [291]:
def gig(starting_table, split_column, target_column):
    

    true_table = starting_table.loc[starting_table[split_column] == 1]
    false_table = starting_table.loc[starting_table[split_column] == 0]
    

    true_counts = true_table[target_column].value_counts()
    false_counts = false_table[target_column].value_counts()  
    starting_counts = starting_table[target_column].value_counts() 
    
    starting_gini = multi_gini(starting_counts)
    true_gini = multi_gini(true_counts)
    false_gini = multi_gini(false_counts)


    starting_size = len(starting_table.index)*1.0
    true_weight = 0.0 if starting_size == 0 else len(true_table.index)/starting_size
    false_weight = 0.0 if starting_size == 0 else len(false_table.index)/starting_size
    

    gig = starting_gini - (true_weight * true_gini + false_weight * false_gini)
    
    return gig
    

In [292]:
def pitch_build_tree_iter(table, choices, target, hypers={} ):

    k = hypers['max-depth'] if 'max-depth' in hypers else min(4, len(choices))
    gig_cutoff = hypers['gig-cutoff'] if 'gig-cutoff' in hypers else 0.0
    
    def iterative_build(k):
        columns_sorted = find_best_splitter(table, choices, target)
        (best_column, gig_value) = columns_sorted[0]
        
        #Note I add _1 or _0 to make it more readable for debugging
        current_paths = [{'conjunction': [(best_column+'_1', build_pred(best_column, 1))],
                          'prediction': None,
                          'gig_score': gig_value},
                         {'conjunction': [(best_column+'_0', build_pred(best_column, 0))],
                          'prediction': None,
                          'gig_score': gig_value}
                        ]
        k -= 1  # we just built a level as seed so subtract 1 from k
        tree_paths = []  # add completed paths here
        
        while k>0:
            new_paths = []
            for path in current_paths:
                conjunct = path['conjunction']  # a list of (name, lambda)
                before_table = generate_table(table, conjunct)  #the subtable the current conjunct leads to
                columns_sorted = find_best_splitter(before_table, choices, target)
                (best_column, gig_value) = columns_sorted[0]
                if gig_value > gig_cutoff:
                    new_path_1 = {'conjunction': conjunct + [(best_column+'_1', build_pred(best_column, 1))],
                                'prediction': None,
                                 'gig_score': gig_value}
                    new_paths.append( new_path_1 ) #true
                    new_path_0 = {'conjunction': conjunct + [(best_column+'_0', build_pred(best_column, 0))],
                                'prediction': None,
                                 'gig_score': gig_value}
                    new_paths.append( new_path_0 ) #false
                else:
                    #not worth splitting so complete the path with a prediction
                    path['prediction'] = pitch_compute_prediction(before_table, target)
                    tree_paths.append(path)
            #end for loop
            
            current_paths = new_paths
            if current_paths != []:
                k -= 1
            else:
                break  # nothing left to extend so have copied all paths to tree_paths
        #end while loop

        #Generate predictions for all paths that have None
        for path in current_paths:
            conjunct = path['conjunction']
            before_table = generate_table(table, conjunct)
            path['prediction'] = pitch_compute_prediction(before_table, target)
            tree_paths.append(path)
        return tree_paths

    return {'paths': iterative_build(k), 'weight': None}

In [293]:
import random
random.seed(1000)
new_seed = seeder(100)
def forest_builder(table, column_choices, target, hypers):
    tree_n = 5 if 'total-trees' not in hypers else hypers['total-trees']
    m = int(len(column_choices)**.5) if 'm' not in hypers else hypers['m']
    k = hypers['max-depth'] if 'max-depth' in hypers else min(2, len(column_choices))
    gig_cutoff = hypers['gig-cutoff'] if 'gig-cutoff' in hypers else 0.0

    #build a single tree - call it multiple times to build multiple trees
    def iterative_build(k):
        train = table.sample(frac=1.0, replace=True, random_state=new_seed())
        train = train.reset_index()
        left_out = table.loc[~table.index.isin(train['index'])]
        left_out = left_out.reset_index() # this gives us the old index in its own column
        oob_list = left_out['index'].tolist()  # list of row indices from original table
        
        rcols = random.sample(column_choices, m)  # subspcace sampling
        columns_sorted = find_best_splitter(train, rcols, target)
        (best_column, gig_value) = columns_sorted[0]

        #Note I add _1 or _0 to make it more readable for debugging
        current_paths = [{'conjunction': [(best_column+'_1', build_pred(best_column, 1))],
                          'prediction': None,
                          'gig_score': gig_value},
                         {'conjunction': [(best_column+'_0', build_pred(best_column, 0))],
                          'prediction': None,
                          'gig_score': gig_value}
                        ]
        k -= 1  # we just built a level as seed so subtract 1 from k
        tree_paths = []  # add completed paths here

        while k>0:
            new_paths = []
            for path in current_paths:
                conjunct = path['conjunction']  # a list of (name, lambda)
                before_table = generate_table(train, conjunct)  #the subtable the current conjunct leads to
                rcols = random.sample(column_choices, m)  # subspace
                columns_sorted = find_best_splitter(before_table, rcols, target)
                (best_column, gig_value) = columns_sorted[0]
                if gig_value > gig_cutoff:
                    new_path_1 = {'conjunction': conjunct + [(best_column+'_1', build_pred(best_column, 1))],
                                'prediction': None,
                                 'gig_score': gig_value}
                    new_paths.append( new_path_1 ) #true
                    new_path_0 = {'conjunction': conjunct + [(best_column+'_0', build_pred(best_column, 0))],
                                'prediction': None,
                                 'gig_score': gig_value
                                 }
                    new_paths.append( new_path_0 ) #false
                else:
                    #not worth splitting so complete the path with a prediction
                    path['prediction'] = pitch_compute_prediction(before_table, target)
                    tree_paths.append(path)
            #end for loop

            current_paths = new_paths
            if current_paths != []:
                k -= 1
            else:
                break  # nothing left to extend so have copied all paths to tree_paths
        #end while loop

        #Generate predictions for all paths that have None
        for path in current_paths:
            conjunct = path['conjunction']
            before_table = generate_table(train, conjunct)
            path['prediction'] = pitch_compute_prediction(before_table, target)
            tree_paths.append(path)
        return (tree_paths, oob_list)
    
    #let's build the forest
    forest = []
    for i in range(tree_n):
        (paths, oob) = iterative_build(k)
        forest.append({'paths': paths, 'weight': None, 'oob': oob})
        
    return forest

# Functions for Accuracy Assesment

For the decision tree, k-fold cross validation will be used to test the accuracy of the different models. For the random forest, I will use out-of-bag error testing.

In [9]:
def k_fold(table, k, target, hypers, candidate_columns):
    name = []
    acc = []
    
    total_len = len(table.index)
    slice_size = int(total_len/(1.0*k))
    slices = []

    #generate the slices
    for i in range(k-1):
        a_slice =  table[i*slice_size:(i+1)*slice_size]
        slices.append( a_slice )
    slices.append( table[(k-1)*slice_size:] )  # whatever is left
    
    #generate test results
    for i in range(k):
        test_table = slices[i]
        train_table = compute_training(slices, i)
        fold_tree = build_tree_iter(train_table, candidate_columns, target, hypers)  # train
        train_table['fold_tree'] = train_table.apply(lambda row: tree_predictor(row,fold_tree),axis=1)
        name.append('fold '+str(i+1)+' test')
        result = 0
        for i in range(len(train_table)):
            if train_table['pitch'].iloc[i] == train_table['fold_tree'].iloc[i]:
                result += 1
        accuracy = float(result)/float(len(train_table))
        acc.append(accuracy)
    
    k_fold_results_table = pd.DataFrame({'accuracy':acc,'name':name}) 
    k_fold_results_table.__doc__ = str(hypers)  # adds comment to remind me of hyper params used
    return k_fold_results_table

In [10]:
import random

#k_fold_random calls k_fold after randomly shuffling the table rows
def k_fold_random(table, k, target, hypers, candidate_columns):
    index_seq = range(0,len(table))
    random.shuffle(index_seq)
    rand_table = pd.DataFrame(columns=table.columns)
    for i in range(0,len(table)):
        rand_table = rand_table.append(table.iloc[index_seq[i]])  
    return k_fold(rand_table, k, target, hypers, candidate_columns)

In [11]:
#Function to get out of bounds table
import numpy as np
def get_oob_table(forest,table):
    oob_list = []
    for i in range(len(forest)):
        oob_list.extend(forest[i]['oob'])
    oob_list = np.array(oob_list)
    oob_table = table.loc[np.unique(oob_list)]
    return oob_table

In [12]:
#function to take votes from the columns that were in the oob table 
def oob_vote_taker(row, forest):
    votes = {0:0, 1:0,2:0,3:0}
    for tree in forest:
        if row.name in tree['oob']:
            prediction = tree_predictor(row, tree)
            votes[prediction] += 1
    winner = 1 if votes[1]>votes[0] else 0  #ties go to 0
    return winner

In [13]:
#function to estimate accuracy for each prediciton 
def pred_accuracy(tree,table):
    result = 0
    for i in range(len(table)):
        if table['pitch'].iloc[i] == table[tree].iloc[i]:
            result += 1
    return float(result)/float(len(table))

# Modelling

I will start by building decision trees of different depths and seeing how accurate they are.

In [16]:
features = ['prior_ball','prior_foul_bunt','prior_foul_tip','prior_hit_by_pitch','prior_hit_into_play',
            'prior_hit_into_play_no_out', 'prior_hit_into_play_score','prior_intent_ball', 'prior_missed_bunt',
            'prior_swinging_strike','prior_swinging_strike_blocked',
            'pk_coors',
            'on_3b','on_2b','on_1b',
            'outs_0','outs_1','outs_2',
            'stand_L','stand_R',
            'strike_0','strike_1','strike_2',
            'ball_0','ball_1','ball_2','ball_3',
            'times_faced_0','times_faced_1','times_faced_2','times_faced_3',
            'pcount_1','pcount_2','pcount_3','pcount_4','pcount_5','pcount_6','pcount_7','pcount_8','pcount_9','pcount_10',
            'pcount_11','pcount_12','pcount_13']

First I will build a tree that is a stump.

In [294]:
tree1 = pitch_build_tree_iter(pitcher,features,'pitch',{'max-depth':1,'gig_cutoff':0.0})

In [295]:
tree1

{'paths': [{'conjunction': [('stand_L_1', <function week4.<lambda>>)],
   'gig_score': 0.007003265294752814,
   'prediction': 0},
  {'conjunction': [('stand_L_0', <function week4.<lambda>>)],
   'gig_score': 0.007003265294752814,
   'prediction': 0}],
 'weight': None}

The tree picks 'stand_L' as the best splitter. This is a pretty good sign. Batter stance has a huge impact on pitch selection.

In [296]:
tree1['paths']

[{'conjunction': [('stand_L_1', <function week4.<lambda>>)],
  'gig_score': 0.007003265294752814,
  'prediction': 0},
 {'conjunction': [('stand_L_0', <function week4.<lambda>>)],
  'gig_score': 0.007003265294752814,
  'prediction': 0}]

In [297]:
pitcher['tree_1'] = pitcher.apply(lambda row: tree_predictor(row,tree1),axis=1)

The tree is predicting fastball everytime. Not good.

In [298]:
pred_accuracy('tree_1',pitcher)

0.5072480053938645

In [126]:
pitcher['tree_1'].value_counts()

0    8899
Name: tree_1, dtype: int64

Now I will try a max depth of 5

In [299]:
tree5 = pitch_build_tree_iter(pitcher,features,'pitch',{'max-depth':5
                                                        ,'gig_cutoff':-10.0})
pitcher['tree_5'] = pitcher.apply(lambda row: tree_predictor(row,tree5),axis=1)

There are now 29 paths.

In [301]:
tree5['paths']


[{'conjunction': [('stand_L_1', <function week4.<lambda>>),
   ('pcount_1_0', <function week4.<lambda>>),
   ('pcount_5_1', <function week4.<lambda>>),
   ('prior_foul_tip_1', <function week4.<lambda>>)],
  'gig_score': 0.007713765997397343,
  'prediction': 3},
 {'conjunction': [('stand_L_0', <function week4.<lambda>>),
   ('outs_1_1', <function week4.<lambda>>),
   ('pcount_8_1', <function week4.<lambda>>),
   ('times_faced_0_1', <function week4.<lambda>>)],
  'gig_score': 0.375,
  'prediction': 0},
 {'conjunction': [('stand_L_0', <function week4.<lambda>>),
   ('outs_1_1', <function week4.<lambda>>),
   ('pcount_8_1', <function week4.<lambda>>),
   ('times_faced_0_0', <function week4.<lambda>>)],
  'gig_score': 0.375,
  'prediction': 1},
 {'conjunction': [('stand_L_1', <function week4.<lambda>>),
   ('pcount_1_1', <function week4.<lambda>>),
   ('times_faced_0_1', <function week4.<lambda>>),
   ('outs_1_1', <function week4.<lambda>>),
   ('prior_hit_into_play_score_1', <function week

The accuracy is not much better than the stump but at least now there are other values being predicted.

In [290]:
pred_accuracy('tree_5',pitcher)

0.5094954489268457

In [165]:
pitcher['tree_5'].value_counts()[0]

8772.0

I will now attempt to find the best tree by testing different depths.

In [66]:
tree10 = pitch_build_tree_iter(pitcher,features,'pitch',{'max-depth':10,'gig_cutoff':0.0})
pitcher['tree_10'] = pitcher.apply(lambda row: tree_predictor(row,tree10),axis=1)
pred_accuracy('tree_10',pitcher)

0.5224182492414878

In [67]:
tree15 = pitch_build_tree_iter(pitcher,features,'pitch',{'max-depth':15,'gig_cutoff':0.0})
pitcher['tree_15'] = pitcher.apply(lambda row: tree_predictor(row,tree15),axis=1)
pred_accuracy('tree_15',pitcher)

0.5339925834363412

In [69]:
tree20 = pitch_build_tree_iter(pitcher,features,'pitch',{'max-depth':20,'gig_cutoff':0.0})
pitcher['tree_20'] = pitcher.apply(lambda row: tree_predictor(row,tree20),axis=1)
pred_accuracy('tree_20',pitcher)

0.5371390043825149

In [65]:
tree44 = pitch_build_tree_iter(pitcher,features,'pitch',{'max-depth':44,'gig_cutoff':0.0})
pitcher['tree_44'] = pitcher.apply(lambda row: tree_predictor(row,tree44),axis=1)
pred_accuracy('tree_44',pitcher)

0.537251376559164

It looks like Tree15 is just about as accurate as the full tree with max depth equaling the number of features. It produces a good variety of predictions as well.

In [71]:
pitcher['tree_15'].value_counts()

0    7518
3     711
2     579
1      91
Name: tree_15, dtype: int64

# Cross Validation

Now lets cross validate the model and see what the accuracy is.

In [74]:
max2_table = k_fold_random(pitcher, 5, 'pitch', {'max-depth':15}, features)
max2_table.head()

,accuracy,name
0,0.513343,fold 1 test
1,0.517556,fold 2 test
2,0.521770,fold 3 test
3,0.512640,fold 4 test
4,0.514615,fold 5 test


# Random Forest

Now I will attempt a random forest to compare it to the normal decision tree.

In [302]:
pitcher_rf = forest_builder(pitcher, features, 'pitch', {'total-trees':50,'max-depth':15})

In [303]:
oob_table = get_oob_table(pitcher_rf,pitcher)
oob_table['forest_1_oob'] = oob_table.apply(lambda row: oob_vote_taker(row, pitcher_rf), axis=1)

The forest figured out the most efficient way is to guess fastball everytime. 

In [305]:
oob_table['forest_1_oob'].value_counts()

0    8899
Name: forest_1_oob, dtype: int64

In [304]:
pred_accuracy('forest_1_oob',oob_table)

0.5072480053938645

In [308]:
import os
home_path =  os.path.expanduser('~')

file_path = '/documents/CS/CIS399/'

file_name = 'greinke_part2_done.csv'

pitcher.to_csv(home_path + file_path + file_name, index=False)